In [1]:
from model import HiFiGAN, MelSpectrogram

In [2]:
model = HiFiGAN(training=False)
mel_spectrogram = MelSpectrogram()


In [3]:
import torch
cp = torch.load('../generator.pth', map_location='cpu')
model.generator.load_state_dict(cp)

<All keys matched successfully>

In [4]:
def generate_audio(mel):
    with torch.no_grad():
        audio = model(mel)
    return audio.squeeze().numpy()

def preprocess_audio(audio):
    mel = mel_spectrogram(torch.from_numpy(audio).unsqueeze(0).float())
    return mel

In [5]:
from datasets import load_dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation", trust_remote_code=True)
sample = ds[0]["audio"]
sample_rate, waveform = sample['sampling_rate'], sample['array']

In [6]:
import IPython.display as ipd
ipd.Audio(waveform, rate=sample_rate)

In [7]:
torch.from_numpy(waveform).unsqueeze(0).shape

torch.Size([1, 93680])

In [8]:
full_audio = []
for i in range(0, len(waveform), 16000):
    audio = generate_audio(preprocess_audio(waveform[i:i+16000]))
    full_audio.extend(audio)

In [9]:
ipd.Audio(full_audio, rate=sample_rate)